In [167]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, code, date, strategy, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": code,
        "date": date,
        "strategy": strategy
    }, data, upsert=True)
    

In [171]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        found.append({
            "code": code,
            "name": item["name"],
            "changes": changes
        })
        data = {
            "code": code,
            "name": item["name"],
            "date": today,
            "strategy": 1,
            "changes": changes
        }
        SaveResult(client, code, today, 1, data)

    if len(found) > 0:
        print pandas.DataFrame(found, columns=["code", "name", "changes"])
    else:
        print("No Result")
        
    client.close()

In [173]:
def Strategy_3():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10),
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    found = found[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 3
        SaveResult(client, item["code"], today, 3, data)
        
    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")
        
    client.close()

In [179]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, item["code"], today, 4, data)
        
    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount", "value"]))

    client.close()

In [172]:
Strategy_1()

      code  name                                         changes
0   002052  同洲电子            [10.0, 9.95850622407, 10.0456621005]
1   601518  吉林高速  [10.0278551532, 10.1226993865, -2.97619047619]
2   300077  国民技术  [9.96533795494, 0.786026200873, 9.99039385207]
3   300293  蓝英装备   [10.027100271, 10.0397614314, -5.09433962264]
4   300333  兆日科技   [10.0087796313, 10.0483091787, 3.08764940239]
5   300366  创意信息   [10.0401606426, 9.98233215548, -7.2131147541]
6   002806  华锋股份            [10.0035473572, 9.98829496684, 10.0]
7   300565  科信技术  [10.0242130751, 10.0159829515, -6.10305152576]
8   300597  吉大通信           [9.97638724911, 10.0, -5.46347452425]
9   300667  必创科技    [9.99405999406, 9.99673309376, 2.2207380197]
10  300718  长盛轴承                    [10.0, 10.0, -6.45002461841]


In [176]:
Strategy_3()

     code    factor   name  price_5  price_today  volume_10  volume_2
0  002931  4.264010   锋龙股份       35        50.80       8155     34773
1  601518  3.603308   吉林高速        3         3.95     108147    389687
2  600853  3.513564   龙建股份        4         5.10      82021    288186
3  000755  3.183396  *ST三维        5         5.27      27187     86547
4  600370  3.114625    三房巷        2         3.05      57867    180234
5  002691  3.043113   冀凯股份       23        25.40      14149     43057
6  600738  3.006270   兰州民百        7         7.63      20575     61854
7  300537  3.005118   广信材料       17        19.27      45522    136799
8  600695  2.777235   绿庭投资        8         8.02      26207     72783
9  002627  2.718123   宜昌交运       18        19.97      14556     39565


In [181]:
Strategy_4()

     code  name    factor     amount      value
0  300634  彩讯股份  0.651501  15.512207  23.809951
1  002864  盘龙药业  0.565316   7.885584  13.948979
2  300504  天邑股份  0.477885  15.139965  31.681163
3  300624  万兴科技  0.419947  10.504566  25.014000
4  300715  凯伦股份  0.411828   4.353898  10.572120
5  300739  明阳电路  0.408891   6.007268  14.691600
6  002930  宏川智慧  0.393257   6.906232  17.561621
7  300730  科创信息  0.389805   4.196161  10.764768
8  002908  德生科技  0.385816   4.810803  12.469160
9  002856  美芝股份  0.369244   2.938925   7.959294


In [208]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    print(pandas.DataFrame(found, columns=["concept", "count", "codes"]))
    
    client.close()

Strategy_2()

  concept  count                                             codes
0    业绩预升      6  [300253, 600486, 002052, 000062, 600708, 002016]
1    股权激励      4                  [002195, 300253, 300054, 300236]
2    国产软件      4                  [002195, 300253, 600756, 600536]
3    5G概念      4                  [300077, 300615, 300597, 300565]
4     云计算      4                  [002195, 300366, 600756, 600536]
5    业绩预降      3                          [002195, 300077, 300348]
